Objective :

We are provided with daily historical sales data of one of the largest Russian software firms - 1C Company. 

The task is to forecast the total amount of products sold in every shop for the test set.We are trying to predict total sales for every product and store in the next month by applying time series algorithm.



In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from scipy.stats import boxcox
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX


pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',1000)

items_df = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/items.csv")
sample_submission_df = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv")
item_categories_df = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv")
sales_train_df = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv")
shops_df = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/shops.csv")
test_df = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/test.csv")


We are provided with data sets of the items and their categories , sales of the items and shops that sells the items.we will go through these data sets and merge them as required.

In [ ]:
items_df.head()

In [ ]:
items_df.shape

In [ ]:
items_df[items_df.duplicated(["item_name"])]

In [ ]:
shops_df.head()

In [ ]:
shops_df.shape

In [ ]:
#displaying the particular items available in each category.
items_df[items_df["item_category_id"]==18]

In [ ]:
items_df.info()

In [ ]:
item_categories_df.head(10)

In [ ]:
item_categories_df.shape

In [ ]:
#Merging the dats sets of items and item categories.
items_df = pd.merge(items_df,item_categories_df,on = "item_category_id",how = "inner")
items_df.head()

In [ ]:
sales_train_df.head()

In [ ]:
sales_train_df.info()

In [ ]:
sales_train_df['date'] =  pd.to_datetime(sales_train_df['date'], format='%d.%M.%Y')
sales_train_df.dtypes


In [ ]:
#records = items_df[items_df['item_category_id'] == 40]

#for index, row in records.iterrows():
     #print(items_df['item_name'])

In [ ]:
#Creating a column that gives the total sales of a particular item.
sales_train_df["item_salevalue"] = sales_train_df["item_price"]*sales_train_df["item_cnt_day"]
#Merging the sales and shops datasets 
sales_train_df = pd.merge(sales_train_df,shops_df,on = "shop_id", how = "inner")
#Merging the sales and items datasets 
sales_train_df = pd.merge(sales_train_df,items_df,on = "item_id", how = "inner")
#Displaying the sales dataset that contains all the required coulmns. 
sales_train_df.head()

In [ ]:
shops_income = sales_train_df.groupby(["shop_id","item_category_name"],as_index = False)["item_salevalue"].sum()
shops_income.rename(columns={'item_salevalue':'total_income'}, inplace=True)
shops_income.sort_values(by=["shop_id"],ascending=True,inplace=True)
shops_income.head()

#### Top Selling Item

In [ ]:
top_selling_item = sales_train_df.groupby(["item_category_id"],as_index = False)["item_cnt_day"].count()
top_selling_item.tail(50)
top_selling_item.sort_values(by=["item_cnt_day"],ascending=False,inplace=True)
top_selling_item.head()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20,8))

sns.barplot(data=top_selling_item, x='item_category_id',y="item_cnt_day")
ax.set_title('', fontweight="bold", fontfamily='serif', fontsize=18)
ax.set(xlabel="item_category_id", ylabel = "item_cnt_day")
ax.patch.set_alpha(0)
plt.show()


#### Top_Selling_Month

In [ ]:
Top_Selling_Month  = sales_train_df.groupby(["date_block_num","item_salevalue"],as_index = False).item_salevalue.max()
#trade_days.sort_values(by=["item_salevalue"],ascending=False,inplace=True)
Top_Selling_Month = Top_Selling_Month.set_index('date_block_num')
Top_Selling_Month.head()


In [ ]:
Top_Selling_Month.plot(figsize=(12, 4))
plt.legend(loc='best')
plt.title('')
plt.show(block=False)

Grouping by shop id and total income and sorting the shops according to the total income.

In [ ]:
shops_income = sales_train_df.groupby(["shop_id"],as_index = False)["item_salevalue"].sum()
shops_income.rename(columns={'item_salevalue':'total_income'}, inplace=True)
shops_income.sort_values(by=["total_income"],ascending=False,inplace=True)
shops_income.head(20)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15,6))

sns.barplot(data=shops_income, x='shop_id',y="total_income")
ax.set_title('TOTAL Income of Shops', fontweight="bold", fontfamily='serif', fontsize=18)
ax.set(xlabel="shop_id", ylabel = "total_income")
ax.patch.set_alpha(0)
plt.show()


It is clear that, shop no 31 has the highest income and lets perform our prediction of sales on shop 31 applying time series models.
Using these models, we try to predict the sales of the next month.

#### Time Series Forecasting 

In [ ]:
#shop_31 = sales_train_df[["date_block_num","item_salevalue"]]
shop_31 = pd.DataFrame(sales_train_df[sales_train_df["shop_id"]==31])
shop_31 = shop_31.groupby(["date_block_num"],as_index = False)["item_salevalue"].sum()
shop_31.head()

In [ ]:
shop_31.shape

In [ ]:
shop_31.plot(figsize=(12, 4))
plt.legend(loc='best')
plt.title('Time Series Plot')
plt.show(block=False)

#### Train-Test Splitting

In [ ]:
train_len = 26
train = shop_31[0:train_len] # first 26 months as train set
test = shop_31[train_len:] # last 8 months as test set

Now, we will build different models and we can choose the best model based on RMSE and MAPE metrics.

#### Naive Method

In [ ]:
y_hat_naive = test.copy()
y_hat_naive['naive_forecast'] = shop_31['item_salevalue'][train_len-1]

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(train['item_salevalue'], label='Train')
plt.plot(test['item_salevalue'], label='Test')
plt.plot(y_hat_naive['naive_forecast'], label='Naive forecast')
plt.legend(loc='best')
plt.title('Naive Method')
plt.show()

#### RMSE and MAPE Calculations 

In [ ]:
rmse = np.sqrt(mean_squared_error(test['item_salevalue'], y_hat_naive['naive_forecast'])).round(2)
mape = np.round(np.mean(np.abs(test['item_salevalue']-y_hat_naive['naive_forecast'])/test['item_salevalue'])*100,2)

results = pd.DataFrame({'Method':['Naive method'], 'MAPE': [mape], 'RMSE': [rmse]})
results = results[['Method', 'RMSE', 'MAPE']]
results

#### Holt Winter's multiplicative method with trend and seasonality

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
y_hat_hwm = test.copy()
model = ExponentialSmoothing(np.asarray(shop_31['item_salevalue']) ,seasonal_periods=12 ,trend='add', seasonal='mul')
model_fit = model.fit(optimized=True)
print(model_fit.params)
y_hat_hwm['hw_forecast'] = model_fit.forecast(8)

In [ ]:
plt.figure(figsize=(12,4))
plt.plot( train['item_salevalue'], label='Train')
plt.plot(test['item_salevalue'], label='Test')
plt.plot(y_hat_hwm['hw_forecast'], label='Holt Winters\'s mulitplicative forecast')
plt.legend(loc='best')
plt.title('Holt Winters\' Mulitplicative Method')
plt.show()

#### RMSE and MAPE Calculations 

In [ ]:
rmse = np.sqrt(mean_squared_error(test['item_salevalue'], y_hat_hwm['hw_forecast'])).round(2)
mape = np.round(np.mean(np.abs(test['item_salevalue']-y_hat_hwm['hw_forecast'])/test['item_salevalue'])*100,2)

tempResults = pd.DataFrame({'Method':['Holt Winters\' multiplicative method'], 'RMSE': [rmse],'MAPE': [mape] })
results = pd.concat([results, tempResults])
results = results[['Method', 'RMSE', 'MAPE']]
results

### Auto-Regression Models

To perform auto-regression models, stationarity and auto_corelation are the two fundamental requirements. We will check these one by one.


Checking the Stationarity of the data set by Adfuller and KPSS tests.

In [ ]:
from statsmodels.tsa.stattools import adfuller
adf_test = adfuller(shop_31['item_salevalue'])

print('ADF Statistic: %f' % adf_test[0])
print('Critical Values @ 0.05: %.2f' % adf_test[4]['5%'])
print('p-value: %f' % adf_test[1])

In [ ]:
from statsmodels.tsa.stattools import kpss
kpss_test = kpss(shop_31['item_salevalue'])

print('KPSS Statistic: %f' % kpss_test[0])
print('Critical Values @ 0.05: %.2f' % kpss_test[3]['5%'])
print('p-value: %f' % kpss_test[1])

#### Box-cox transformation

In [ ]:
#Performing Box-cox transformation to get the data set stationary.
from scipy.stats import boxcox
shop_31_boxcox = pd.Series(boxcox(shop_31['item_salevalue'], lmbda=0), index = shop_31.index)

plt.figure(figsize=(12,4))
plt.plot(shop_31_boxcox, label='After Box Cox tranformation')
plt.legend(loc='best')
plt.title('After Box Cox transform')
plt.show()

In [ ]:
shop31_boxcox_diff = pd.Series(shop_31_boxcox - shop_31_boxcox.shift(), shop_31.index)
plt.figure(figsize=(12,4))
plt.plot(shop31_boxcox_diff, label='After Box Cox tranformation and differencing')
plt.legend(loc='best')
plt.title('After Box Cox transform and differencing')
plt.show()

Checking the p-values after transformation.

In [ ]:
shop31_boxcox_diff.dropna(inplace=True)

In [ ]:
adf_test = adfuller(shop31_boxcox_diff)

print('ADF Statistic: %f' % adf_test[0])
print('Critical Values @ 0.05: %.2f' % adf_test[4]['5%'])
print('p-value: %f' % adf_test[1])

In [ ]:
kpss_test = kpss(shop31_boxcox_diff)

print('KPSS Statistic: %f' % kpss_test[0])
print('Critical Values @ 0.05: %.2f' % kpss_test[3]['5%'])
print('p-value: %f' % kpss_test[1])

#### Plotting ACF and PACF plots.

In [ ]:
plt.figure(figsize=(12,4))
plot_acf(shop_31["item_salevalue"], ax=plt.gca(), lags = 15)
plt.show()

In [ ]:
plt.figure(figsize=(12,4))
plot_pacf(shop_31["item_salevalue"], ax=plt.gca(), lags = 15)
plt.show()

In [ ]:
train_shop31_boxcox = shop_31_boxcox[:train_len]
test_shop31_boxcox = shop_31_boxcox[train_len:]
train_shop31_boxcox_diff = shop31_boxcox_diff[:train_len-1]
test_shop31_boxcox_diff = shop31_boxcox_diff[train_len-1:]

### ARIMA Model

In [ ]:
model = ARIMA(train_shop31_boxcox_diff, order=(1, 0, 1)) 
model_fit = model.fit()
print(model_fit.params)

In [ ]:
y_hat_ar = shop31_boxcox_diff.copy()
y_hat_ar['ar_forecast_boxcox_diff'] = model_fit.predict(shop31_boxcox_diff.index.min(), shop31_boxcox_diff.index.max())
y_hat_ar['ar_forecast_boxcox'] = y_hat_ar['ar_forecast_boxcox_diff'].cumsum()
y_hat_ar['ar_forecast_boxcox'] = y_hat_ar['ar_forecast_boxcox'].add(shop_31_boxcox[0])
y_hat_ar['ar_forecast'] = np.exp(y_hat_ar['ar_forecast_boxcox'])

#### Plotting train, test and forecast 

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(train['item_salevalue'], label='Train')
plt.plot(test['item_salevalue'], label='Test')
plt.plot(y_hat_ar['ar_forecast'][test.index.min():], label='Auto regression forecast')
plt.legend(loc='best')
plt.title('Auto Regression Method')
plt.show()

#### RMSE and MAPE Calculations 

In [ ]:
rmse = np.sqrt(mean_squared_error(test['item_salevalue'], y_hat_ar['ar_forecast'][test.index])).round(2)
mape = np.round(np.mean(np.abs(test['item_salevalue']-y_hat_ar['ar_forecast'][test.index.min():])/test['item_salevalue'])*100,2)

tempResults = pd.DataFrame({'Method':['Autoregressive (AR) method'], 'RMSE': [rmse],'MAPE': [mape] })
results = pd.concat([results, tempResults])
results = results[['Method', 'RMSE', 'MAPE']]
results

### MOVING AVERAGE

In [ ]:
model = ARIMA(train_shop31_boxcox_diff, order=(0, 0, 1)) 
model_fit = model.fit()
print(model_fit.params)

In [ ]:
y_hat_ma = shop31_boxcox_diff.copy()
y_hat_ma['ma_forecast_boxcox_diff'] = model_fit.predict(shop31_boxcox_diff.index.min(), shop31_boxcox_diff.index.max())
y_hat_ma['ma_forecast_boxcox'] = y_hat_ma['ma_forecast_boxcox_diff'].cumsum()
y_hat_ma['ma_forecast_boxcox'] = y_hat_ma['ma_forecast_boxcox'].add(shop_31_boxcox[0])
y_hat_ma['ma_forecast'] = np.exp(y_hat_ma['ma_forecast_boxcox'])

#### Plotting train, test and forecast 

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(shop_31['item_salevalue'][:train_len], label='Train')
plt.plot(shop_31['item_salevalue'][train_len:], label='Test')
plt.plot(y_hat_ma['ma_forecast'][test.index.min():], label='Moving average forecast')
plt.legend(loc='best')
plt.title('Moving Average Method')
plt.show()

#### RMSE and MAPE Calculations 

In [ ]:
rmse = np.sqrt(mean_squared_error(test['item_salevalue'], y_hat_ma['ma_forecast'][test.index])).round(2)
mape = np.round(np.mean(np.abs(test['item_salevalue']-y_hat_ma['ma_forecast'][test.index.min():])/test['item_salevalue'])*100,2)

tempResults = pd.DataFrame({'Method':['Moving Average (MA) method'], 'RMSE': [rmse],'MAPE': [mape] })
results = pd.concat([results, tempResults])
results = results[['Method', 'RMSE', 'MAPE']]
results

#### SARIMAX 

In [ ]:
model = SARIMAX(train_shop31_boxcox, order=(1, 0, 1), seasonal_order=(1, 0, 1, 15)) 
model_fit = model.fit()
print(model_fit.params)

In [ ]:
y_hat_sarima = shop31_boxcox_diff.copy()
y_hat_sarima['sarima_forecast_boxcox'] = model_fit.predict(shop_31["item_salevalue"].index.min(), shop_31["item_salevalue"].index.max())
y_hat_sarima['sarima_forecast'] = np.exp(y_hat_sarima['sarima_forecast_boxcox'])

#### Plotting train, test and forecast 

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(train['item_salevalue'], label='Train')
plt.plot(test['item_salevalue'], label='Test')
plt.plot(y_hat_sarima['sarima_forecast'][test.index.min():], label='SARIMA forecast')
plt.legend(loc='best')
plt.title('Seasonal autoregressive integrated moving average (SARIMA) method')
plt.show()

#### RMSE and MAPE Calculations 

In [ ]:
rmse = np.sqrt(mean_squared_error(test['item_salevalue'], y_hat_sarima['sarima_forecast'][test.index.min():])).round(2)
mape = np.round(np.mean(np.abs(test['item_salevalue']-y_hat_sarima['sarima_forecast'][test.index.min():])/test['item_salevalue'])*100,2)

tempResults = pd.DataFrame({'Method':['Seasonal autoregressive integrated moving average (SARIMA) method'], 'RMSE': [rmse],'MAPE': [mape] })
results = pd.concat([results, tempResults])
results = results[['Method', 'RMSE', 'MAPE']]
results